# Translation Group


In this submodule, we build up all the pieces of translation group, from a cluster and a lattice to primitive crystal and its supercell. This allows us to extract critical information of the structures, manipulate them, and study the electron and vibrational properties of them. 

In [1]:
import numpy as np
np.set_printoptions(linewidth=180, suppress=True)

## Cluster

We begin with a cluster which consists of a collection of atoms in arranged positions.

For example, we can create a square of H atoms by providing the atom positions and orbitals:

In [2]:
from principia_materia.translation_group import Cluster
from principia_materia.symmetry import PointGroup

In [3]:
square = Cluster(atoms={
    "H": [
        [ 1,  1],
        [ 1, -1],
        [-1,  1],
        [-1, -1],
    ]
}, orbitals="s")
square

In [4]:
square.orbitals

[(0, 's'), (1, 's'), (2, 's'), (3, 's')]

The cluster is centered at the origin.

In [5]:
square.center

array([0., 0.])

We can rotate the cluster using the point operators in the C4v point group. The atoms will be rotated into each other's spots but the entire cluster remains invariant.
This will be used to construct representations of a cluster.

In [6]:
pg = PointGroup("C4v")

In [7]:
square.rotate_atoms(pg.G["c4z"][:2, :][:, :2])

array([[ 1., -1.],
       [-1., -1.],
       [ 1.,  1.],
       [-1.,  1.]])

## Lattice

The `Lattice` class contains only the lattice vectors and optionaly the point group of the lattice.
The class mainly serves as an abstract class to build into more complex structures.
But it contain all the key properties of a lattice, including the volume of the unit lattice, lengths and angels of the latiice vectors, reciprocal lattice, etc. Conversion between lattice basis and cartesian basis can also be done using the methods `lattice_to_cartesian` and `cartesian_to_lattice`. 

In [8]:
from principia_materia.translation_group import Lattice

In [9]:
lattice = Lattice(vec=np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
]))

In [10]:
lattice.lattice_to_cartesian([0.5, 0.5, 0])

array([0.25, 0.25, 0.5 ])

## LatticeFTG

Derived from `Lattice`, `LatticeFTG` explores properties of a superlattice with a supercell matrix.
It finds the lattice points that fit in the given supercell.
If point group is provided, irreducible lattice points and the rotational mapping of the lattice points will be found.


### The algorithm to find lattice points fit in the supercell
Algorithm used in finding the lattice points in the superlattice is described in Appendix D of paper: PHYSICAL REVIEW B 100, 014303 (2019). 
It is the mathamatically optimal algorithm to find these points, with $\mathcal{O}(n)$ time complexity. 
Meanwhile, it is also possible to assign and compute the index of each found point in $\mathcal{O}(1)$ time complexity.
This makes non-diagonal supercells as efficient to work with as diagonal supercells.

In [11]:
from principia_materia.translation_group import LatticeFTG

In [12]:
superlattice = LatticeFTG(vec=np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
]),
  supa=np.array([
      [-1, 1, 1],
      [1, -1, 1],
      [1, 1, -1],
]),
  pg="Oh",
)

In [13]:
superlattice.lattice_points

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])

Multiplicity is the size of the supercell.

In [14]:
superlattice.multiplicity

4

The irreducible lattice points form a unique set lattice points under point group symmetry.
The `irreducible_lattice_points_trans` attribute contains the point group operations that
rotate the lattice points to the irreducible lattice points.

In [15]:
superlattice.irreducible_lattice_points

array([[0, 0, 0],
       [0, 0, 1]])

In [16]:
superlattice.irreducible_lattice_points_trans

array(['E', 'E', 'Ic2e', 'Ic2c'], dtype='<U7')

## Kpoints

`Kpoint` class finds all the k-points that fit for a given supercell, and the irreducible k-points and the point group operations that transform k-points to their irreducible counterpart.
It uses `LatticeFTG` to find the reciprocal lattice points and convert them into kpoints.

In [17]:
from principia_materia.translation_group import Kpoints

In [18]:
kpoints = Kpoints(vec=np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.0, 0.5],
    [0.5, 0.5, 0.0],
]),
  supa=np.array([
      [-1, 1, 1],
      [1, -1, 1],
      [1, 1, -1],
]),
  pg="Oh",
)

In [25]:
kpoints.kpoints

array([[Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)],
       [Fraction(1, 2), Fraction(1, 2), Fraction(0, 1)],
       [Fraction(1, 2), Fraction(0, 1), Fraction(1, 2)],
       [Fraction(0, 1), Fraction(1, 2), Fraction(1, 2)]], dtype=object)

In [27]:
kpoints.irreducible_kpoints

array([[Fraction(0, 1), Fraction(0, 1), Fraction(0, 1)],
       [Fraction(1, 2), Fraction(1, 2), Fraction(0, 1)]], dtype=object)

In [28]:
kpoints.irreducible_kpoints_trans

array(['E', 'E', 'Ic2e', 'Ic2c'], dtype='<U7')

## CrystalFTG

Derived from `Crystal` class which is a combination of `Lattice` and `Cluster`, this class contains critical information of a crystal structure, including lattice vectors, species of atoms and their positions and the supercell matrix. 
The instances of this class store the key information of the strutures we will be studying with this package. 

In [19]:
from principia_materia.translation_group import CrystalFTG

Here we define a rocksalt structure with lattice constant of 1.0, with a $2\times2\times2$ diagonal supercell.

In [20]:
structure =  CrystalFTG(
    vec=1 * np.array([
        [0.0, 0.5, 0.5],
        [0.5, 0.0, 0.5],
        [0.5, 0.5, 0.0],
    ]),
    atoms={
        "Na": [[0, 0, 0]],
        "Cl": [[0.5, 0.5, 0.5]],
    },
    orbitals="p",
    supa=np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2],
    ])
)

In [21]:
structure.to_dict()

OrderedDict([('vec',
              array([[0. , 0.5, 0.5],
                     [0.5, 0. , 0.5],
                     [0.5, 0.5, 0. ]])),
             ('atoms',
              OrderedDict([('Na', array([[0., 0., 0.]])),
                           ('Cl', array([[0.5, 0.5, 0.5]]))])),
             ('orbitals', 'p'),
             ('supa',
              array([[2, 0, 0],
                     [0, 2, 0],
                     [0, 0, 2]]))])

The supercell creates a finite translation group with a finite number of translation points, within which we can explore vibrational properties of the crystal.

In [22]:
structure.supa_lattice.lattice_points

array([[0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 1, 0],
       [0, 0, 1],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 1]])

1. Displacement basis of q-points

    The orbitals are set to "p" which transform like atomic displacements.
    We then can compute the displacement basis at an arbitrary q-point.
    The data is in dictionary form, with keys denoting the index of atom and the orbital, 
    and the values being the displacement vector of the basis.

In [23]:
structure.get_basis_at_q(np.array([0.5, 0.0, 0.0]))

OrderedDict([((0, 'p_x'),
              array([[ 0.35355339+0.j,  0.        +0.j,  0.        +0.j],
                     [-0.35355339-0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.35355339+0.j,  0.        +0.j,  0.        +0.j],
                     [-0.35355339-0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.35355339+0.j,  0.        +0.j,  0.        +0.j],
                     [-0.35355339-0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.35355339+0.j,  0.        +0.j,  0.        +0.j],
                     [-0.35355339-0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.        +0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.        +0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.        +0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.        +0.j,  0.        +0.j,  0.        +0.j],
                     [ 0.        +0.j,  0.        +0.j,  0.        +0.j],
            

2. Wigner Seitz cell

    One of the important properties of finite translation group is Wigner Seitz cell, 
    which puts every atom closest to the origin. 
    This feature is crucial in the Fourier Interpolation of phonons and their interactions.
    
    The method return 3 pieces of information:
        1. The translation point mapping from the atom of the WS cell to the primitive cell.
        2. The index of the atom of WS cell in the primitive cell.
        3. The WS weight of the atom.

In [24]:
ws_cell_translation, ws_cell_atoms, ws_cell_weights = \
    structure.get_wigner_seitz_cell(center=np.array([0, 0, 0]))

`CrystalFTG` builds supercell based on the primitive cell and the supercell matrix:

In [30]:
structure.primitive, structure.supa

(<principia_materia.translation_group.crystal.Crystal at 0x7f9ebf4a30d0>,
 array([[2, 0, 0],
        [0, 2, 0],
        [0, 0, 2]]))

## Find a minimum supercell that fits the given k-point(s)

Function `get_minimum_supercell` returns a minimum supercell that fits the provided k-point(s). The algorithm is decribed in Section III(B) of paper: PHYSICAL REVIEW B 100, 014303 (2019). 

Note that the mininmum supercellcell is not unique. Here we find the Hermite Normal form of the minimum cell which is unique.

In [35]:
from principia_materia import Fraction
from principia_materia.io_interface import parse_array
from principia_materia.translation_group import get_minimum_supercell

In [38]:
qpts = parse_array("1/2 0 0; 0 1/2 0", Fraction)
supa = get_minimum_supercell(qpts)
qpts, supa

([[Fraction(1, 2), Fraction(0, 1), Fraction(0, 1)],
  [Fraction(0, 1), Fraction(1, 2), Fraction(0, 1)]],
 array([[2, 0, 0],
        [0, 2, 0],
        [0, 0, 1]]))